In [16]:
## 아래 패키지 설치
!pip install pymysql
!pip show sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00
Name: SQLAlchemy
Version: 2.0.39
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: greenlet, typing-extensions
Required-by: ipython-sql, langchain


### 라이브러리 선언

In [17]:
import pandas as pd

In [2]:
# prompt: 구글드라이브 연동해줘

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/data_analysis_2025/03. 데이터 불러오기


### 일반 파일 불러오기 (로컬)

In [18]:
csData = pd.read_csv("../dataset/customerdata.csv")

In [19]:
csData.head(2)

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.90000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.44195,2,4,2.682023,Sleeping-dog


### github 공유된 파일 불러오는것도 가능하다

In [20]:
carDataPath = "https://raw.githubusercontent.com/hyokwan/python-lecture/refs/heads/master/dataset/cars.csv"

In [21]:
carsDf = pd.read_csv(carDataPath)

### 공공데이터포털 데이터 불러오기

In [22]:
pirPath = "https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000002999024&fileDetailSn=1&dataNm=%ED%95%9C%EA%B5%AD%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B3%B5%EC%82%AC_%EC%A3%BC%ED%83%9D%EA%B8%88%EC%9C%B5%EA%B4%80%EB%A0%A8%20%EC%A7%80%EC%88%98_20160101"
pirDf = pd.read_csv( pirPath, encoding = "ms949" )
pirDf.tail(2)

,연도,전국소득대비 주택가격 비율,서울소득대비 주택가격 비율,부산소득대비 주택가격 비율,대구소득대비 주택가격 비율,인천소득대비 주택가격 비율,광주소득대비 주택가격 비율,대전소득대비 주택가격 비율,울산소득대비 주택가격 비율,세종소득대비 주택가격 비율,...,세종평균 대출금액 평균 연소득,경기평균 대출금액 평균 연소득,강원평균 대출금액 평균 연소득,충북평균 대출금액 평균 연소득,충남평균 대출금액 평균 연소득,전북평균 대출금액 평균 연소득,전남평균 대출금액 평균 연소득,경북평균 대출금액 평균 연소득,경남평균 대출금액 평균 연소득,제주평균 대출금액 평균 연소득
141,2015,5.68,7.18,5.78,6.48,5.76,5.46,5.17,5.11,5.22,...,2.17,2.56,1.95,2.11,2.02,1.88,1.67,1.99,2.14,2.54
142,2016,4.31,7.30,5.73,6.88,5.74,4.92,4.79,5.38,5.76,...,2.39,2.38,1.74,1.83,1.81,1.63,1.75,1.71,2.05,2.29


### 데이터프레임 (웹_로컬파일_데이터베이스) 자료형 데이터베이스에 저장하기

In [26]:
csData = pd.read_csv("../dataset/customer.csv")
csData["new_col"] = 324
from sqlalchemy import create_engine
### 데이터베이스 접속정보
user = "hkcode"
password = "fintech1308!"
dbip = "my8003.gabiadb.com"
dbport = "3306"
database = "hkcodedb"
dbConPath = f"mysql+pymysql://{user}:{password}@{dbip}:{dbport}/{database}"
dbConPath
myengine = create_engine(dbConPath)
# 데이터 저장하기
csData.to_sql(name = "ANAL_PIRDATA_김효관", con=myengine, index=False, if_exists="replace")

20000

### 저장한 데이터 확인하기

In [28]:
# 데이터 불러오기
csData = pd.read_sql_query(sql="SELECT * FROM ANAL_PIRDATA_김효관", con=myengine)
csData.head(2)

,balance,stock,label,new_col
0,30000000,22500000,normal,324
1,280000000,48000000,diamond,324


In [29]:
type(csData)

pandas.core.frame.DataFrame

###퀴즈 데이터셋 폴더내 kopo_product_volume0-3패턴 데이터 불러오기

### 실습 링크에서 확인가능

In [30]:
import pandas as pd
from sqlalchemy import create_engine

### 폴더에서 kopo_product_volume.csv 불러오기

# kopo_product_volume.csv 파일을 불러와서 저장할 리스트
kopoFileList = []
# 에러발생 파일 저장할 리스트
kopoErrorList = []
# encoding 리스트
encodings = ['utf-8', 'cp949']

### 데이터베이스 접속정보
user = "hkcode"
password = "fintech1308!"
dbip = "my8003.gabiadb.com"
dbport = "3306"
database = "hkcodedb"
dbConPath = f"mysql+pymysql://{user}:{password}@{dbip}:{dbport}/{database}"
myengine = create_engine(dbConPath)

# 파일 불러오기
for i in range(0, 4):
    kopo_Path = '../dataset/'
    fileName = f"kopo_product_volume{i}.csv"
    table_name = fileName.split('.')
    for j in range(0,1):
        try:
            eachFile = pd.read_csv(kopo_Path+fileName, encoding=encodings[j])
            ### 데이터 저장하기
            eachFile.to_sql(name = table_name[0]+'_한정석', con=myengine, index=False, if_exists="replace")

        except Exception as e:
            print(e)
            kopoErrorList.append(kopo_Path)

### 데이터 불러오기
for i in range(0,4):
    try:
        loadData = pd.read_sql_query(sql=f"SELECT * FROM kopo_product_volume{i}_한정석", con=myengine)
        kopoFileList.append(loadData)
    except Exception as e:
        print(e)

[Errno 2] No such file or directory: '../dataset/kopo_product_volume1.csv'
[Errno 2] No such file or directory: '../dataset/kopo_product_volume3.csv'
(pymysql.err.ProgrammingError) (1146, "Table 'hkcodedb.kopo_product_volume1_한정석' doesn't exist")
[SQL: SELECT * FROM kopo_product_volume1_한정석]
(Background on this error at: https://sqlalche.me/e/20/f405)
(pymysql.err.ProgrammingError) (1146, "Table 'hkcodedb.kopo_product_volume3_한정석' doesn't exist")
[SQL: SELECT * FROM kopo_product_volume3_한정석]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [31]:
myengine.dispose()